# VIC - Introduction to Visual Computing

**Instructor:** Maria Vakalopoulou\
**T.A.:** Joseph Boyd\
**Date:** December 16, 2022

## Image segmentation

For the lab exercises use `jupyter` or `colab` (or anything else at your own risk). It can be installed by issuing `pip install jupyter` on Linux after installing pip itself with `apt install python-pip`. Mac and Windows users can find the installation instructions online. The first thing that is needed is to import some libraries, such as `numpy`, `scipy`, `random` and `matplotlib` for visualization. If they are missing you can simply install them with e.g., `pip install numpy`

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Otsu's method

Otsu's method performs a binary segmentation of the image into two classes (foregound and background) by choosing an intensity threshold $T \in \{0, \ldots, L-1\}$ for the $L$ image greylevels, so as to maximise the inter-class variance,

$$T = \arg\max_{t} w_0(t)\cdot w_1(t)\cdot [\mu_0(t) - \mu_1(t)]^2,$$

where the $w_0(t)$ and $w_1(t)$ are the class probabilities for a given choice of $t$,

$$w_0(t) = \sum_{i=0}^{t-1}p(i)$$

$$w_1(t) = \sum_{i=t}^{L-1}p(i),$$

and the class means,

$$\mu_0(t) = \frac{\sum_{i=0}^{t-1}ip(i)}{w_0(t)}$$

$$\mu_1(t) = \frac{\sum_{i=t}^{L-1}ip(i)}{w_1(t)}$$

First let's load an image into memory:

In [ ]:
from skimage.data import page

img = page()

fig, ax = plt.subplots(figsize=(10, 5))
ax.imshow(img, cmap='gray')

First, compute the histogram of image intensity values using `numpy.bincount` and plot using `matplotlib.pyplot.plot`:

In [ ]:
# TODO

Compute the vector of class probabilities $w_0(t)$ and $w_1(t)$ for all $t$:

In [ ]:
w_0 = # TODO
w_1 = # TODO

Compute the vector of class means $\mu_0(t)$ and $\mu_1(t)$ for all $t$:

In [ ]:
mu_0 = # TODO
mu_1 = # TODO

Now iterate over all intensity values to find the optimal threshold `T`:

In [ ]:
# TODO

In [ ]:
from ipywidgets import interact, widgets
from IPython.display import display

hist = np.bincount(img.flatten())
masks = [img > threshold for threshold in range(255)]

def f_display(threshold):
    fig, axes = plt.subplots(figsize=(15, 3), ncols=2)
    axes[0].imshow(masks[threshold], cmap='gray')
    axes[1].plot(hist, color='blue')
    axes[1].axvline(threshold, color='red', linestyle='--')

interact(f_display, threshold=widgets.IntSlider(min=0, max=255, step=1, value=T));

# Watershed algorithm

The watershed algorithm can be used for separating objects in an image. From a set of initial markers, it simulates a "flooding" of the image topology by growing outwards from the markers until regions intersect. The resulting boundaries between regions are known as *watershed lines*.

We will apply the watershed algorithm as part of a segmentation pipeline for separating cells in a fluorescence microscopy image taken from the [2018 Data Science Bowl](https://bbbc.broadinstitute.org/BBBC038) dataset.

First, let's load the image and perform a Gaussian blur for denoising:

In [ ]:
from skimage.io import imread
from skimage.filters import gaussian

img = imread('cells.png')

fig, axes = plt.subplots(figsize=(15, 10), ncols=2)
axes[0].imshow(img, cmap='gray')

filtered_img = # TODO

axes[1].imshow(filtered_img, cmap='gray')

Now, apply a threshold using Otsu's method. You may use the `skimage` library function, or your own from above.

In [ ]:
from skimage.filters import threshold_otsu

segmentation = # TODO

fig, axes = plt.subplots(figsize=(15, 10), ncols=2)
axes[0].imshow(img, cmap='gray')
axes[1].imshow(segmentation, cmap='gray')

The main challenge for the watershed algorithm is automatically selecting the flooding markers. A common approach is to select the local maxima of the image distance map. For a given binary segmentation, the distance map is an image indicating the shortest distance (whether Euclidean or other) of each foreground pixel to the background. Use `scipy.ndimage.distance_transform_edt` for compute the distance map for the above segmentation:

In [ ]:
from scipy.ndimage import distance_transform_edt

distance_map = # TODO

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(distance_map, cmap='gray')

Use the `skimage.feature.peak_local_max` to compute maxima for `distance_map`. You will need to adjust the `min_distance` and `footprint` parameters to find a balance between over- and under-segmentation. **N.B** the aim is have one marker per cell. Then, create a `local_maxima` image of the same size as `img`, that is non-zero only at the marker points. Finally, we use `skimage.measure.label` to attribute a unique value for each label, which we then visualise.

In [ ]:
from skimage.feature import peak_local_max
from skimage.measure import label
from skimage.color import label2rgb

local_maxima # TODO

# N.B. local_maxima should be an image!
markers = label(local_maxima)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(label2rgb(markers, bg_label=0))

Apply the function `skimage.segmentation.watershed` to `distance_map` using the `markers` compute above, as well as at the binary `segmentation` as the `mask` parameter. Set `watershed_line=True` for a cleaner visualisation. **N.B.** By convention, the watershed "floods" from bottom-to-top. Therefore, the *inverse* (negative) distance map is used.

In [ ]:
from skimage.segmentation import watershed

labels = # TODO

fig, axes = plt.subplots(figsize=(15, 10), ncols=2)

axes[0].imshow(img, cmap='gray')
axes[1].imshow(labels)

## SLIC

Simple linear iterative clustering (SLIC) is an algorithm for computing *superpixels*, an intermediary between pixels and objects. Superpixels are groups of pixels that cluster both by proximity and by colour properties.

Let's begin by loading an image:

In [ ]:
from skimage import data

img = data.astronaut()

fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(img)

Use `skimage.segmentation.slic` to compute superpixels for the above image. You should tune the parameters `n_segments`, the number of superpixels, and `compactness`, the greater the compactness, the more spatial the clustering.

In [ ]:
from skimage.segmentation import slic

segments = # TODO

fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(segments)

Run the code below to visualise the superpixel contours:

In [ ]:
from skimage.measure import find_contours

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img)

for i in range(np.max(segments)):
    cont = find_contours(segments == i)[0]
    ax.fill(cont[:, 1], cont[:, 0], facecolor='none', edgecolor='blue')

Now let's try and code SLIC from scratch! SLIC represents each pixel $p_k$ as a 5-dimensional vector,

$$p_i = [l_i, a_i, b_i, x_i, y_i],$$

where $l_i, a_i, b_i$ are the values from the CIELAB transformation of the RGB colour space, and $x_i, y_i$ are the spatial coordinates of $p_i$.

First, compute these features for each pixel in a $512 \times 512 \times 5$ `numpy.array`:

In [ ]:
from skimage.color import rgb2rgbcie

# TODO

SLIC iteratively updates cluster centers, the centroids of the superpixels. Initially, these are allocated according to a regular grid, controlled by a spatial paramters $S$. **N.B.** that the initial superpixels centroids should be "perturbed" by shifting them to the pixel of lowest gradient in a $3\times3$ window, but we leave this as optional for now.

In [ ]:
# TODO

For each cluster, one examines all pixels $p_i$ in a $2S \times 2S$ window around the cluster $k$ centroid, $p_k = [l_k, a_k, b_k, x_k, y_k]$. Pixels are assigned the cluster minimising the distance,

$$D_s = d_{lab} + \frac{m}{S}d_{xy},$$

where $d_{lab} = \sqrt{(l_k - l_i)^2 + (a_k - a_i)^2 + (b_k - b_i)^2}$, and $d_{xy} = \sqrt{(x_k - x_i)^2 + (y_k - y_i)^2}$. Once all pixels have been assigned, new cluster centroids are computed as the element-wise mean of all member pixels. The algorithm iterates until centroid locations are unchanged within a certain tolerance under an $L_1$ error.

In [ ]:
# TODO

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img)

for i in range(np.max(segments)):
    cont = find_contours(segments == i)[0]
    ax.fill(cont[:, 1], cont[:, 0], facecolor='none', edgecolor='blue')